# PCA año 1990 

In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
datos_1990 = pd.read_csv('Marginacion_1990.csv')

In [5]:
datos_1990.head()

,CVE_ENT,NOM_ENT,POB_TOT,ANALF,SPRIM,OVSDE,OVSEE,OVSAE,VHAC,OVPT,PL.5000,PO2SM,IM,GM,LUGAR,A.O
0,1,Aguascalientes,719659,7.06,33.85,10.88,4.97,4.18,51.03,7.45,26.97,62.53,-0.89,Bajo,27,1990
1,2,Baja California,1660855,4.68,24.01,4.89,10.49,19.59,45.43,8.13,11.95,39.96,-1.34,Muy bajo,30,1990
2,3,Baja California Sur,317764,5.39,27.77,7.03,11.10,10.17,48.60,13.99,25.60,54.06,-0.97,Bajo,28,1990
3,4,Campeche,535185,15.40,44.82,24.80,15.02,29.52,65.03,24.16,36.21,68.07,0.48,Alto,10,1990
4,5,Coahuila de Zaragoza,1972340,5.48,28.18,10.81,5.24,7.76,50.00,8.48,17.13,60.89,-1.05,Bajo,29,1990


Escogemos únicamente los indicadores socioeconómicos 

In [8]:
variables = datos_1990[['ANALF', 'SPRIM', 'OVSDE', 'OVSEE', 
                        'OVSAE', 'VHAC', 'OVPT', 'PL.5000','PO2SM']]

Para realizar análisis de componentes prinicpales necesitamos la matriz de varianzas y covarianzas

In [45]:
var_cov = np.cov(variables.T)
pd.DataFrame(var_cov)

,0,1,2,3,4,5,6,7,8
0,48.050971,62.344852,79.252754,42.175038,70.018710,44.762822,79.201723,95.911630,43.653863
1,62.344852,102.910295,116.808847,60.922886,101.710010,62.068374,104.954850,160.578301,63.872228
2,79.252754,116.808847,187.266276,69.815556,117.214819,78.409546,128.112779,208.962209,95.847230
3,42.175038,60.922886,69.815556,62.553974,82.242282,36.548285,78.848831,104.723978,40.936536
4,70.018710,101.710010,117.214819,82.242282,158.912195,68.153079,120.806449,169.655422,75.189166
5,44.762822,62.068374,78.409546,36.548285,68.153079,56.955756,74.955112,103.355823,46.159634
6,79.201723,104.954850,128.112779,78.848831,120.806449,74.955112,156.613515,168.670446,66.134650
7,95.911630,160.578301,208.962209,104.723978,169.655422,103.355823,168.670446,311.286645,108.794833
8,43.653863,63.872228,95.847230,40.936536,75.189166,46.159634,66.134650,108.794833,86.705362


In [47]:
def compute_Jacobi_rotation(Ak, idx1, idx2):
    """
    Helper function for computing entries of Jacobi rotation.
    Argumentss:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        egval (array): matrix with Ak's eigenvalues in the main diagonal.
        s (float): matrix with Ak's eigenvectors in its columns.
    """
    if np.abs(Ak[idx1,idx2]) > np.finfo(float).eps:
        tau = (Ak[idx2, idx2] - Ak[idx1, idx1])/(2*Ak[idx1, idx2])
        if tau >= 0:
            s = 1
        else:
            s = -1
        t_star = s/(np.abs(tau) + np.sqrt(1+tau**2))
        c = 1/np.sqrt(1+t_star**2)
        s = c*t_star
    else: 
        c = 1
        s = 0
    J = np.array([[c, s], [-s, c]])
    Q = np.identity(J.shape[0])
    egval = J.T@Ak@J
    egvec = Q@J
    return (egval, egvec)

In [48]:
compute_Jacobi_rotation(var_cov,0,1)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 9 is different from 2)

In [65]:
def compute_Jacobi_rotation(Ak, idx1, idx2):
    """
    Compute Jacobi rotation matrix.
    Args:
        Ak (numpy ndarray): Matrix of iteration k in Jacobi rotation method.
        idx1 (int): index for rows in Jacobi rotation matrix.
        idx2 (int): index for columns in Jacobi rotation matrix.
    Returns:
        J (numpy ndarray): Jacobi rotation matrix.
    """
    c,s = compute_cos_sin_Jacobi_rotation(Ak, idx1, idx2)
    m,n = Ak.shape
    J = np.eye(m)
    J[idx1, idx1] = J[idx2, idx2] = c
    J[idx1, idx2] = s
    J[idx2, idx1] = -s
    return J

# Sweep 1

índices: 

         (1,2),(1,3),(1,4),(1,5),(1,6),(1,7), (1,8),(1,9),

         (2,3),(2,4),(2,5), (2,6), (2,7), (2,8), (2,9)
         
         (3,4), (3,5), (3,6), (3,7), (3,9), (3,9),
         
         (4,5), (4,6), (4,7), (4,8), (4,9),
         
         (5,6), (5,7), (5,8), (5,9),
         
         (6,7), (6,8), (6,9),
         
         (7,8), (7,9),
         
         (8,9)
         

In [71]:
J_1_2 = compute_Jacobi_rotation(var_cov,0,1)

In [72]:
A_1_2 = J_1_2.T@var_cov@J_1_2

In [73]:
pd.DataFrame(A_1_2)

,0,1,2,3,4,5,6,7,8
0,7.368468e+00,-7.453743e-16,2.537806,2.027034,3.055822,3.568237,8.973074,-7.430133,1.653742
1,-4.102209e-16,1.435928e+02,141.134210,74.069042,123.443136,76.442534,131.178952,186.893618,77.347181
2,2.537806e+00,1.411342e+02,187.266276,69.815556,117.214819,78.409546,128.112779,208.962209,95.847230
3,2.027034e+00,7.406904e+01,69.815556,62.553974,82.242282,36.548285,78.848831,104.723978,40.936536
4,3.055822e+00,1.234431e+02,117.214819,82.242282,158.912195,68.153079,120.806449,169.655422,75.189166
5,3.568237e+00,7.644253e+01,78.409546,36.548285,68.153079,56.955756,74.955112,103.355823,46.159634
6,8.973074e+00,1.311790e+02,128.112779,78.848831,120.806449,74.955112,156.613515,168.670446,66.134650
7,-7.430133e+00,1.868936e+02,208.962209,104.723978,169.655422,103.355823,168.670446,311.286645,108.794833
8,1.653742e+00,7.734718e+01,95.847230,40.936536,75.189166,46.159634,66.134650,108.794833,86.705362


In [74]:
J_1_3 = compute_Jacobi_rotation(A_1_2,0,2)

In [75]:
A_1_3 = J_1_3.T@A_1_2@J_1_3
pd.DataFrame(A_1_3)

,0,1,2,3,4,5,6,7,8
0,7.332675e+00,-1.990376,3.299779e-16,1.042243,1.402471,2.462095,7.165443,-10.376330,0.301870
1,-1.990376e+00,143.592798,1.411202e+02,74.069042,123.443136,76.442534,131.178952,186.893618,77.347181
2,-1.721557e-17,141.120174,1.873021e+02,69.837199,117.246258,78.452070,128.226583,208.836643,95.861021
3,1.042243e+00,74.069042,6.983720e+01,62.553974,82.242282,36.548285,78.848831,104.723978,40.936536
4,1.402471e+00,123.443136,1.172463e+02,82.242282,158.912195,68.153079,120.806449,169.655422,75.189166
5,2.462095e+00,76.442534,7.845207e+01,36.548285,68.153079,56.955756,74.955112,103.355823,46.159634
6,7.165443e+00,131.178952,1.282266e+02,78.848831,120.806449,74.955112,156.613515,168.670446,66.134650
7,-1.037633e+01,186.893618,2.088366e+02,104.723978,169.655422,103.355823,168.670446,311.286645,108.794833
8,3.018704e-01,77.347181,9.586102e+01,40.936536,75.189166,46.159634,66.134650,108.794833,86.705362


In [77]:
J_1_4 = compute_Jacobi_rotation(A_1_3,0,3)
A_1_4 = J_1_4.T@A_1_3@J_1_4
pd.DataFrame(A_1_4)

,0,1,2,3,4,5,6,7,8
0,7.313010e+00,-3.387250,-1.317399,-7.421326e-17,-0.149185,1.772215,5.676775,-12.349981,-0.470404
1,-3.387250e+00,143.592798,141.120174,7.401832e+01,123.443136,76.442534,131.178952,186.893618,77.347181
2,-1.317399e+00,141.120174,187.302069,6.982477e+01,117.246258,78.452070,128.226583,208.836643,95.861021
3,-2.826263e-16,74.018316,69.824773,6.257364e+01,82.254104,36.588227,78.969969,104.509606,40.934947
4,-1.491852e-01,123.443136,117.246258,8.225410e+01,158.912195,68.153079,120.806449,169.655422,75.189166
5,1.772215e+00,76.442534,78.452070,3.658823e+01,68.153079,56.955756,74.955112,103.355823,46.159634
6,5.676775e+00,131.178952,128.226583,7.896997e+01,120.806449,74.955112,156.613515,168.670446,66.134650
7,-1.234998e+01,186.893618,208.836643,1.045096e+02,169.655422,103.355823,168.670446,311.286645,108.794833
8,-4.704041e-01,77.347181,95.861021,4.093495e+01,75.189166,46.159634,66.134650,108.794833,86.705362


In [78]:
J_1_5 = compute_Jacobi_rotation(A_1_4,0,4)
A_1_5 = J_1_5.T@A_1_4@J_1_5
pd.DataFrame(A_1_5)

,0,1,2,3,4,5,6,7,8
0,7.312863e+00,-3.265771,-1.202019,0.080944,-7.820503e-18,1.839282,5.795655,-12.183022,-0.396412
1,-3.265771e+00,143.592798,141.120174,74.018316,1.234464e+02,76.442534,131.178952,186.893618,77.347181
2,-1.202019e+00,141.120174,187.302069,69.824773,1.172475e+02,78.452070,128.226583,208.836643,95.861021
3,8.094424e-02,74.018316,69.824773,62.573638,8.225406e+01,36.588227,78.969969,104.509606,40.934947
4,1.771283e-17,123.446410,117.247497,82.254065,1.589123e+02,68.151302,120.800804,169.667494,75.189593
5,1.839282e+00,76.442534,78.452070,36.588227,6.815130e+01,56.955756,74.955112,103.355823,46.159634
6,5.795655e+00,131.178952,128.226583,78.969969,1.208008e+02,74.955112,156.613515,168.670446,66.134650
7,-1.218302e+01,186.893618,208.836643,104.509606,1.696675e+02,103.355823,168.670446,311.286645,108.794833
8,-3.964121e-01,77.347181,95.861021,40.934947,7.518959e+01,46.159634,66.134650,108.794833,86.705362


In [ ]:
# Índice (1,6) en python (0,5)